In [1]:
import yfinance as yf

# Télécharger les données pour des indices économiques
vix = yf.download('^VIX', interval='1h', start="2023-01-01", end="2023-12-31")  # Volatilité implicite
bond_yields = yf.download('^IRX', interval='1h', start="2023-01-01", end="2023-12-31")  # Taux des bons du Trésor US à 3 mois

/Users/dabereabasse/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
vix.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-01-03 02:00:00-06:00,23.090000,23.110001,22.99,23.000000,23.000000,0
2023-01-03 03:00:00-06:00,22.990000,23.010000,22.74,22.790001,22.790001,0
2023-01-03 04:00:00-06:00,22.790001,22.950001,22.77,22.920000,22.920000,0
2023-01-03 05:00:00-06:00,22.930000,22.930000,22.83,22.910000,22.910000,0
2023-01-03 06:00:00-06:00,22.910000,23.280001,22.90,23.190001,23.190001,0


In [3]:
bond_yields.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-01-03 07:20:00-06:00,4.260,4.260,4.250,4.250,4.250,0
2023-01-03 08:20:00-06:00,4.250,4.260,4.250,4.258,4.258,0
2023-01-03 09:20:00-06:00,4.258,4.258,4.225,4.243,4.243,0
2023-01-03 10:20:00-06:00,4.243,4.248,4.228,4.248,4.248,0
2023-01-03 11:20:00-06:00,4.248,4.258,4.238,4.248,4.248,0


In [26]:
from wbdata import get_dataframe
import datetime

# Sélectionner des indicateurs (par exemple, PIB, taux de chômage)
indicators = {'NY.GDP.MKTP.CD': 'GDP', 'SL.UEM.TOTL.ZS': 'Unemployment', 'SP.POP.TOTL': 'Population'}

# Télécharger les données de 2020 à 2023 pour les États-Unis
data = get_dataframe(indicators, country='USA', date=(datetime.datetime(2010, 1, 1), datetime.datetime(2023, 12, 31)))
print(data.head())


               GDP  Unemployment   Population
date                                         
2023  2.736094e+13         3.625  334914895.0
2022  2.574411e+13         3.650  333271411.0
2021  2.359403e+13         5.349  332048977.0
2020  2.132295e+13         8.055  331526933.0
2019  2.152140e+13         3.669  328329953.0
